In [ ]:
import pandas as pd
import pandas_datareader as reader
import datetime as dt
import numpy as np
import statsmodels.api as sm

In [ ]:
df = pd.read_csv("/content/russell-1000-index-02-09-2021.csv")
df.drop(df.tail(1).index,inplace=True)
df

,Symbol,Name,Last,Change,%Chg,Open,High,Low,Volume,Time
0,A,Agilent Technologies,123.41,0.23,+0.19%,123.92,125.74,122.57,2065300.0,02/08/21
1,AAL,American Airlines Gp,17.77,0.58,+3.37%,17.32,18.09,17.24,44177500.0,02/08/21
2,AAP,Advance Auto Parts Inc,158.44,1.16,+0.74%,158.38,160.02,157.56,957800.0,02/08/21
3,AAPL,Apple Inc,136.91,0.15,+0.11%,136.03,136.96,134.92,71297195.0,02/08/21
4,ABBV,Abbvie Inc,106.84,-1.89,-1.74%,108.89,108.98,105.81,9201101.0,02/08/21
...,...,...,...,...,...,...,...,...,...,...
995,XPO,Xpo Logistics Inc,120.47,2.53,+2.15%,118.03,120.95,118.03,878700.0,02/08/21
996,XRAY,Dentsply Sirona Inc,55.88,0.64,+1.16%,55.41,56.25,55.20,915000.0,02/08/21
997,XRX,Xerox Corp,24.24,0.74,+3.15%,23.65,24.36,23.65,2629200.0,02/08/21
998,XYL,Xylem Inc,100.96,2.77,+2.82%,98.42,101.33,98.41,1228600.0,02/08/21


In [ ]:
success = []
failures = []
feb_first = pd.Timestamp('2016-02-01')
start = feb_first-pd.DateOffset(years=4)
novem_17 = feb_first-pd.DateOffset(months=3)
novem_18 = pd.Timestamp("2016-11-01")
may = pd.Timestamp("2016-05-01")
year = pd.Timestamp("2017-02-01")
half = feb_first+pd.DateOffset(months=6)
fama = reader.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start, year)[0][1:]
fama = fama/100
fama

,Mkt-RF,SMB,HML,RF
Date,,,,
2012-02-02,0.0017,0.0040,0.0000,0.00000
2012-02-03,0.0158,0.0059,0.0041,0.00000
2012-02-06,-0.0005,-0.0026,-0.0022,0.00000
2012-02-07,0.0016,-0.0027,-0.0010,0.00000
2012-02-08,0.0025,-0.0016,0.0046,0.00000
...,...,...,...,...
2017-01-26,-0.0010,-0.0062,0.0039,0.00002
2017-01-27,-0.0012,0.0001,-0.0067,0.00002
2017-01-30,-0.0068,-0.0071,-0.0044,0.00002


In [ ]:
for i in range(len(df.index)):
    ticker = df.iloc[i]['Symbol']
    try:
        prices = reader.get_data_yahoo(ticker, start=start, end=year)['Close']
        if prices.index[0] >= start + pd.Timedelta(days=10):
            raise ValueError('Not old enough')
        daily_return = np.log((prices/prices.shift(1)).dropna())
        daily_return = pd.merge(daily_return, fama, on='Date')
        daily_return['y'] = daily_return['Close'] - daily_return['RF']
    
    except:
        failures.append(ticker)
        print(i, ticker, "failure")
        continue

    train = daily_return[:novem_17-pd.Timedelta(days=1)]
    test_1 = daily_return[novem_17:feb_first]
    test_2 = daily_return[feb_first:may]
    test_3 = daily_return[feb_first:novem_18]
    test_4 = daily_return[feb_first:half]
    test_5 = daily_return[feb_first:year]

    train_y = train['y']
    train_X = train[["Mkt-RF", "SMB", "HML"]]
    model = sm.OLS(train_y, train_X)
    results = model.fit()
    
    test_X1 = test_1[["Mkt-RF", "SMB", "HML"]]
    test_y1 = test_1['y']
    preds1 = results.predict(test_X1)
    abnormal1 = (test_y1 - preds1).sum()

    test_X2 = test_2[["Mkt-RF", "SMB", "HML"]]
    test_y2 = test_2['y']
    preds2 = results.predict(test_X2)
    abnormal2 = (test_y2 - preds2).sum()

    test_X3 = test_3[["Mkt-RF", "SMB", "HML"]]
    test_y3 = test_3['y']
    preds3 = results.predict(test_X3)
    abnormal3 = (test_y3 - preds3).sum()

    test_X4 = test_4[["Mkt-RF", "SMB", "HML"]]
    test_y4 = test_4['y']
    preds4 = results.predict(test_X4)
    abnormal4 = (test_y4 - preds4).sum()

    test_X5 = test_5[["Mkt-RF", "SMB", "HML"]]
    test_y5 = test_5['y']
    preds5 = results.predict(test_X5)
    abnormal5 = (test_y5 - preds5).sum()

    price_1 = prices[novem_17+pd.Timedelta(days=1)]
    price_2 = prices[feb_first]
    price_3 = prices[may+pd.Timedelta(days=1)]
    price_4 = prices[novem_18]
    price_5 = prices[half]
    price_6 = prices[year]

    success.append((ticker, abnormal1, abnormal2, abnormal3, abnormal4, abnormal5,price_1,
                    price_2, price_3, price_4, price_5, price_6))
    print(i, ticker, "success")

0 A success
1 AAL success
2 AAP success
3 AAPL success
4 ABBV failure
5 ABC success
6 ABMD success
7 ABT success
8 ACAD success
9 ACC success
10 ACGL success
11 ACHC success
12 ACI failure
13 ACM success
14 ACN success
15 ADBE success
16 ADI success
17 ADM success
18 ADP success
19 ADPT failure
20 ADS success
21 ADSK success
22 ADT failure
23 AEE success
24 AEP success
25 AES success
26 AFG success
27 AFL success
28 AGCO success
29 AGIO failure
30 AGNC success
31 AGO success
32 AGR failure
33 AIG success
34 AIRC failure
35 AIV success
36 AIZ success
37 AJG success
38 AKAM success
39 AL success
40 ALB success
41 ALGM failure
42 ALGN success
43 ALK success
44 ALKS success
45 ALL success
46 ALLE failure
47 ALLY failure
48 ALNY success
49 ALSN failure
50 ALXN success
51 AM failure
52 AMAT success
53 AMCR failure
54 AMD success
55 AME success
56 AMED success
57 AMG success
58 AMGN success
59 AMH failure
60 AMP success
61 AMT success
62 AMWL failure
63 AMZN success
64 AN success
65 ANAT succ

In [ ]:
success_df = pd.DataFrame(success,  columns=['Ticker', '-3', '+3', '+9', '+6', '+12','Nov 17', 'Feb 18', 'May 18', 'Nov 18', 'half year', 'year'])
success_df.to_excel("output_2016_2.xlsx")

In [ ]:
failures